<a href="https://colab.research.google.com/github/Maagnitude/coupon-recommend-models/blob/main/coupon_recommend_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **In-vehicle Coupon Recommendation Models**
Importing the necessary libraries.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import model_selection
from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt

## **Uploading the dataset as a csv file**

Ανεβάζουμε το dataset μέσω του **github url**, απ το **repository**. Έτσι γλιτώνουμε το mount του drive.

In [2]:
url = "https://raw.githubusercontent.com/Maagnitude/coupon_recommend_models/main/in-vehicle-coupon-recommendation.csv"
df = pd.read_csv(url)
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


## **Another way to load the dataset (Drive mount)**

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# from google.colab import drive
# drive.mount('/content/drive/')
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/in-vehicle-coupon-recommendation.csv')

## **Here we see the shape of the dataframe**. 

We have a **12684 x 26** table. 12684 entries, 26 attributes.

In [3]:
df.shape

(12684, 26)

With the use of **df.info()**, we can see that there are some missing. Car has only 108 non-null values, out of 12684. So we will drop this attribute for sure.We can also see clearly the attributes with null values, using **df.isnull().sum()**.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

In [5]:
df.isnull().sum()

destination                 0
passanger                   0
weather                     0
temperature                 0
time                        0
coupon                      0
expiration                  0
gender                      0
age                         0
maritalStatus               0
has_children                0
education                   0
occupation                  0
income                      0
car                     12576
Bar                       107
CoffeeHouse               217
CarryAway                 151
RestaurantLessThan20      130
Restaurant20To50          189
toCoupon_GEQ5min            0
toCoupon_GEQ15min           0
toCoupon_GEQ25min           0
direction_same              0
direction_opp               0
Y                           0
dtype: int64

In [6]:
df= df.drop("car", axis='columns')

Here we check the value counts for each categorical value. It seems like a good strategy for each **null-value** to get the value of the **most frequent**.

In [7]:
df['Bar'].value_counts()

never    5197
less1    3482
1~3      2473
4~8      1076
gt8       349
Name: Bar, dtype: int64

In [8]:
df['CoffeeHouse'].value_counts()

less1    3385
1~3      3225
never    2962
4~8      1784
gt8      1111
Name: CoffeeHouse, dtype: int64

In [9]:
df['CarryAway'].value_counts()

1~3      4672
4~8      4258
less1    1856
gt8      1594
never     153
Name: CarryAway, dtype: int64

In [10]:
df['RestaurantLessThan20'].value_counts()

1~3      5376
4~8      3580
less1    2093
gt8      1285
never     220
Name: RestaurantLessThan20, dtype: int64

In [11]:
df['Restaurant20To50'].value_counts()

less1    6077
1~3      3290
never    2136
4~8       728
gt8       264
Name: Restaurant20To50, dtype: int64

# **Impute NaN values**
Done by using the most frequent of the existing values.

In [12]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['Bar'] = imputer.fit_transform(df['Bar'].values.reshape(-1,1))[:,0]

In [13]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['CoffeeHouse'] = imputer.fit_transform(df['CoffeeHouse'].values.reshape(-1,1))[:,0]

In [14]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['CarryAway'] = imputer.fit_transform(df['CarryAway'].values.reshape(-1,1))[:,0]

In [15]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['RestaurantLessThan20'] = imputer.fit_transform(df['RestaurantLessThan20'].values.reshape(-1,1))[:,0]

In [16]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df['Restaurant20To50'] = imputer.fit_transform(df['Restaurant20To50'].values.reshape(-1,1))[:,0]

In [17]:
df

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,55,6PM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,1
12680,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12681,Work,Alone,Snowy,30,7AM,Coffee House,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,0
12682,Work,Alone,Snowy,30,7AM,Bar,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,1,1,0,1,0


Now, as we see, there is no null-value in our dataframe.

In [18]:
df.isnull().sum()

destination             0
passanger               0
weather                 0
temperature             0
time                    0
coupon                  0
expiration              0
gender                  0
age                     0
maritalStatus           0
has_children            0
education               0
occupation              0
income                  0
Bar                     0
CoffeeHouse             0
CarryAway               0
RestaurantLessThan20    0
Restaurant20To50        0
toCoupon_GEQ5min        0
toCoupon_GEQ15min       0
toCoupon_GEQ25min       0
direction_same          0
direction_opp           0
Y                       0
dtype: int64

# **ONE-HOT ENCODING**

Here we can see the data type of each attribute. Temperature has only **three** numerical values, so we will cast it as **categorical** in order to one-hot encode it later.

In [19]:
df.dtypes

destination             object
passanger               object
weather                 object
temperature              int64
time                    object
coupon                  object
expiration              object
gender                  object
age                     object
maritalStatus           object
has_children             int64
education               object
occupation              object
income                  object
Bar                     object
CoffeeHouse             object
CarryAway               object
RestaurantLessThan20    object
Restaurant20To50        object
toCoupon_GEQ5min         int64
toCoupon_GEQ15min        int64
toCoupon_GEQ25min        int64
direction_same           int64
direction_opp            int64
Y                        int64
dtype: object

In [ ]:
df['temperature'].value_counts()

80    6528
55    3840
30    2316
Name: temperature, dtype: int64

In [20]:
df['temperature']=pd.Categorical(df.temperature)

In [21]:
df

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,55,6PM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,1
12680,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12681,Work,Alone,Snowy,30,7AM,Coffee House,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,0
12682,Work,Alone,Snowy,30,7AM,Bar,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,1,1,0,1,0


Numerical-value attributes are already in one-hot encoded form (0s and 1s), so we won't mess with them. 

In [22]:
df['occupation'].value_counts()

Unemployed                                   1870
Student                                      1584
Computer & Mathematical                      1408
Sales & Related                              1093
Education&Training&Library                    943
Management                                    838
Office & Administrative Support               639
Arts Design Entertainment Sports & Media      629
Business & Financial                          544
Retired                                       495
Food Preparation & Serving Related            298
Healthcare Practitioners & Technical          244
Healthcare Support                            242
Community & Social Services                   241
Legal                                         219
Transportation & Material Moving              218
Architecture & Engineering                    175
Personal Care & Service                       175
Protective Service                            175
Life Physical Social Science                  170


In [23]:
dummydf = df['occupation'].str.get_dummies(sep='&')
dummydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0    Administrative Support            12684 non-null  int64
 1    Engineering                       12684 non-null  int64
 2    Extraction                        12684 non-null  int64
 3    Financial                         12684 non-null  int64
 4    Forestry                          12684 non-null  int64
 5    Grounds Cleaning                  12684 non-null  int64
 6    Maintenance                       12684 non-null  int64
 7    Material Moving                   12684 non-null  int64
 8    Mathematical                      12684 non-null  int64
 9    Media                             12684 non-null  int64
 10   Related                           12684 non-null  int64
 11   Repair                            12684 non-null  int64
 12   Service          

In [24]:
dummydf['Arts']=dummydf['Arts Design Entertainment Sports ']
dummydf['Design']=dummydf['Arts Design Entertainment Sports ']
dummydf['Arts']=dummydf['Arts Design Entertainment Sports ']
dummydf['Entertainment']=dummydf['Arts Design Entertainment Sports ']
dummydf['Social Science']=dummydf['Life Physical Social Science']
dummydf['Physical Science']=dummydf['Life Physical Social Science']
dummydf['Life Science']=dummydf['Life Physical Social Science']
dummydf.drop(['Arts Design Entertainment Sports ','Life Physical Social Science'],axis=1,inplace=True)  
dummydf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0    Administrative Support    12684 non-null  int64
 1    Engineering               12684 non-null  int64
 2    Extraction                12684 non-null  int64
 3    Financial                 12684 non-null  int64
 4    Forestry                  12684 non-null  int64
 5    Grounds Cleaning          12684 non-null  int64
 6    Maintenance               12684 non-null  int64
 7    Material Moving           12684 non-null  int64
 8    Mathematical              12684 non-null  int64
 9    Media                     12684 non-null  int64
 10   Related                   12684 non-null  int64
 11   Repair                    12684 non-null  int64
 12   Service                   12684 non-null  int64
 13   Serving Related           12684 non-null  int64
 14   Social Services      

In [25]:
df

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,1~3,4~8,1~3,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,55,6PM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,1
12680,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12681,Work,Alone,Snowy,30,7AM,Coffee House,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,0
12682,Work,Alone,Snowy,30,7AM,Bar,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,1,1,0,1,0


In [26]:
df = pd.concat([df, dummydf], axis=1)
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,Student,Training,Transportation,Unemployed,Arts,Design,Entertainment,Social Science,Physical Science,Life Science
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0


In [27]:
df.drop(['occupation'],axis=1,inplace=True)
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,Student,Training,Transportation,Unemployed,Arts,Design,Entertainment,Social Science,Physical Science,Life Science
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,0,0,0,1,0,0,0,0,0,0


In [28]:
df.columns

Index(['destination', 'passanger', 'weather', 'temperature', 'time', 'coupon',
       'expiration', 'gender', 'age', 'maritalStatus', 'has_children',
       'education', 'income', 'Bar', 'CoffeeHouse', 'CarryAway',
       'RestaurantLessThan20', 'Restaurant20To50', 'toCoupon_GEQ5min',
       'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same',
       'direction_opp', 'Y', ' Administrative Support', ' Engineering',
       ' Extraction', ' Financial', ' Forestry', ' Grounds Cleaning ',
       ' Maintenance', ' Material Moving', ' Mathematical', ' Media',
       ' Related', ' Repair', ' Service', ' Serving Related',
       ' Social Services', ' Technical', 'Architecture ', 'Building ',
       'Business ', 'Community ', 'Computer ', 'Construction ', 'Education',
       'Farming Fishing ', 'Food Preparation ', 'Healthcare Practitioners ',
       'Healthcare Support', 'Installation Maintenance ', 'Legal', 'Library',
       'Management', 'Office ', 'Personal Care ', 'Production Occupat

In [29]:
df_categorical=df.select_dtypes(exclude='number')

In [30]:
df_categorical

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,education,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,Some college - no degree,$37500 - $49999,never,never,1~3,4~8,1~3
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,Some college - no degree,$37500 - $49999,never,never,1~3,4~8,1~3
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,Some college - no degree,$37500 - $49999,never,never,1~3,4~8,1~3
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,Some college - no degree,$37500 - $49999,never,never,1~3,4~8,1~3
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,Some college - no degree,$37500 - $49999,never,never,1~3,4~8,1~3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,Home,Partner,Rainy,55,6PM,Carry out & Take away,1d,Male,26,Single,Bachelors degree,$75000 - $87499,never,never,1~3,4~8,1~3
12680,Work,Alone,Rainy,55,7AM,Carry out & Take away,1d,Male,26,Single,Bachelors degree,$75000 - $87499,never,never,1~3,4~8,1~3
12681,Work,Alone,Snowy,30,7AM,Coffee House,1d,Male,26,Single,Bachelors degree,$75000 - $87499,never,never,1~3,4~8,1~3
12682,Work,Alone,Snowy,30,7AM,Bar,1d,Male,26,Single,Bachelors degree,$75000 - $87499,never,never,1~3,4~8,1~3


In [31]:
for value in df_categorical:
  df= pd.get_dummies(df,columns=[value])

In [32]:
df

,has_children,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,Administrative Support,Engineering,Extraction,...,RestaurantLessThan20_1~3,RestaurantLessThan20_4~8,RestaurantLessThan20_gt8,RestaurantLessThan20_less1,RestaurantLessThan20_never,Restaurant20To50_1~3,Restaurant20To50_4~8,Restaurant20To50_gt8,Restaurant20To50_less1,Restaurant20To50_never
0,1,1,0,0,0,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,1,1,1,0,0,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,1,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12680,0,1,0,0,0,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12681,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12682,0,1,1,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [33]:
df.shape

(12684, 134)

## **Dropping Duplicate Values**

In [34]:
df.drop_duplicates(inplace=True)
df.shape

(12610, 134)

We can see that 'toCoupon_GEQ5min' gets only one value. So we don't need that. 

In [35]:
df.select_dtypes('int64').nunique()

has_children                 2
toCoupon_GEQ5min             1
toCoupon_GEQ15min            2
toCoupon_GEQ25min            2
direction_same               2
direction_opp                2
Y                            2
 Administrative Support      2
 Engineering                 2
 Extraction                  2
 Financial                   2
 Forestry                    2
 Grounds Cleaning            2
 Maintenance                 2
 Material Moving             2
 Mathematical                2
 Media                       2
 Related                     2
 Repair                      2
 Service                     2
 Serving Related             2
 Social Services             2
 Technical                   2
Architecture                 2
Building                     2
Business                     2
Community                    2
Computer                     2
Construction                 2
Education                    2
Farming Fishing              2
Food Preparation             2
Healthca

In [36]:
df.drop(columns=['toCoupon_GEQ5min'], inplace=True)

In [37]:
df.describe()

,has_children,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,Administrative Support,Engineering,Extraction,Financial,...,RestaurantLessThan20_1~3,RestaurantLessThan20_4~8,RestaurantLessThan20_gt8,RestaurantLessThan20_less1,RestaurantLessThan20_never,Restaurant20To50_1~3,Restaurant20To50_4~8,Restaurant20To50_gt8,Restaurant20To50_less1,Restaurant20To50_never
count,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,...,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000,12610.000000
mean,0.414512,0.559794,0.116019,0.215543,0.784457,0.567565,0.050595,0.013878,0.012213,0.043061,...,0.434972,0.281761,0.101665,0.164235,0.017367,0.259001,0.057732,0.020936,0.494052,0.168279
std,0.492657,0.496432,0.320260,0.411215,0.411215,0.495434,0.219177,0.116989,0.109838,0.203003,...,0.495773,0.449875,0.302220,0.370503,0.130640,0.438103,0.233245,0.143175,0.499984,0.374129
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
df

,has_children,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,Administrative Support,Engineering,Extraction,Financial,...,RestaurantLessThan20_1~3,RestaurantLessThan20_4~8,RestaurantLessThan20_gt8,RestaurantLessThan20_less1,RestaurantLessThan20_never,Restaurant20To50_1~3,Restaurant20To50_4~8,Restaurant20To50_gt8,Restaurant20To50_less1,Restaurant20To50_never
0,1,0,0,0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,1,1,0,0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,1,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12679,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12680,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12681,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12682,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [39]:
df.dtypes

has_children              int64
toCoupon_GEQ15min         int64
toCoupon_GEQ25min         int64
direction_same            int64
direction_opp             int64
                          ...  
Restaurant20To50_1~3      uint8
Restaurant20To50_4~8      uint8
Restaurant20To50_gt8      uint8
Restaurant20To50_less1    uint8
Restaurant20To50_never    uint8
Length: 133, dtype: object

In [40]:
# plt.figure(figsize=(25, 15))

# for i, column in enumerate(df.columns):
#   plt.subplot(4, 6, i + 1)
#   sns.boxplot(data=df[column])
#   plt.title(column)

# plt.tight_layout()
# plt.show()

#**Classification (Logistic Regression)**

In [100]:
X = df.drop('Y', axis=1)

In [101]:
y = df['Y']

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [103]:
lr = linear_model.LogisticRegression()
lr_mod = lr.fit(X_train, y_train)

# **Baseline Logistic Regression: 0.691**

In [104]:
print(f"Baseline Logistic Regression: {round(lr_mod.score(X_test, y_test), 3)}")

Baseline Logistic Regression: 0.691


# **Classification (KNN Algorithm)**

In [99]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

neighbors =  5   < 20   < 30 
accuracy  = 0.65 < 0.69 < 0.71


In [114]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 30, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=30)

In [117]:
y_pred = classifier.predict(X_test)

In [118]:
accuracy = classifier.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.71


# **SVM Algorithm**

Αργεί πάρα πολύ η εκπαίδευση του.

In [119]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [120]:
SVM = SVC(kernel="rbf", random_state=None, probability=True, cache_size=200, gamma=0.1)
SVM.fit(X_train, y_train)

SVC(gamma=0.1, probability=True)

In [121]:
y_pred_SVM = SVM.predict(X_test)
accuracy2 = SVM.score(X_test, y_test)

print(f'Accuracy: {accuracy2:.2f}')

Accuracy: 0.75


# **DECISION TREE**

In [122]:
DTC = DecisionTreeClassifier().fit(X_train, y_train)

#Predict the response for test dataset
y_pred_DTC = DTC.predict(X_test)

accuracy3 = SVM.score(X_test, y_test)

print(f'Accuracy: {accuracy3:.2f}')

Accuracy: 0.75


# **Random Forest**

In [123]:
from sklearn.ensemble import RandomForestClassifier

randForest = RandomForestClassifier()

randForest.fit(X_train, y_train)
accuracy4 = randForest.score(X_test, y_test)
print(f'Accuracy: {accuracy4:.2f}')

Accuracy: 0.75


# **Naive Bayes**

In [125]:
from sklearn.naive_bayes import GaussianNB

nayes = GaussianNB()
nayes.fit(X_train, y_train)
accuracy5 = nayes.score(X_test, y_test)
print(f'Accuracy: {accuracy5:.2f}')

Accuracy: 0.62


# **Clustering**